# IndoML 2023 Tuturial: Part 2
## At Sesame street

### Plan:

1. We will use Huggingface transformers for finetuning pretrained models.
2. This will allow us to easily swap out models to find the most suitable one.
    * This is achieved using the `AutoModel*` and `AutoTokenizer` classes.
3. It also comes with various new libraries integrated for efficient fine-tuning, 
multi-gpu or distributed training, and more.

### Methods that we will try:

1. DistilBERT
2. mBERT/distilmBERT (multilingual)
3. XLM-RoBERTa (multilingual)
4. DeBERTa (multilingual)

### Efficient Training

1. Linear Probing
2. PEFT
3. Distilled Models

### Libraries to explore

1. 🤗 Transfomers > Trainer
2. 🤗 Evaluate
2. wandb

## Load `dataset`

In [1]:
from tqdm import tqdm
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

tqdm.pandas()

dataset = load_dataset("AmazonScience/massive")


/home/bsantra/miniconda3/envs/py38/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-08-25 12:45:31.499697: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-25 12:45:32.251859: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


## Load `AutoTokenizer`

In [2]:
checkpoint = "bert-base-multilingual-cased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)


def tokenize_function(example):
    return tokenizer(example["utt"], truncation=True)



## Preprocessing

### Using the Trainer class

We will be using the HF Trainer class to handle a lot of the boilerplate tasks. This Trainer class expects certain format for the dataset.

1. Tokenize the dataset
2. A "labels" column should exits

In [3]:
tokenized_datasets = dataset.map(tokenize_function, batched=True)

In [4]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [5]:
tokenized_datasets['train'][0].keys()

dict_keys(['id', 'locale', 'partition', 'scenario', 'intent', 'utt', 'annot_utt', 'worker_id', 'slot_method', 'judgments', 'input_ids', 'attention_mask'])

In [6]:
# Remove columns that are not needed
to_remove = list(tokenized_datasets['train'][0].keys())

# Don't remove these columns
to_remove.remove('input_ids')
to_remove.remove('attention_mask')
to_remove.remove('intent')

tokenized_datasets = tokenized_datasets.remove_columns(to_remove)
tokenized_datasets = tokenized_datasets.rename_column('intent', 'labels')


In [7]:
tokenized_datasets['train'].features

{'labels': ClassLabel(names=['datetime_query', 'iot_hue_lightchange', 'transport_ticket', 'takeaway_query', 'qa_stock', 'general_greet', 'recommendation_events', 'music_dislikeness', 'iot_wemo_off', 'cooking_recipe', 'qa_currency', 'transport_traffic', 'general_quirky', 'weather_query', 'audio_volume_up', 'email_addcontact', 'takeaway_order', 'email_querycontact', 'iot_hue_lightup', 'recommendation_locations', 'play_audiobook', 'lists_createoradd', 'news_query', 'alarm_query', 'iot_wemo_on', 'general_joke', 'qa_definition', 'social_query', 'music_settings', 'audio_volume_other', 'calendar_remove', 'iot_hue_lightdim', 'calendar_query', 'email_sendemail', 'iot_cleaning', 'audio_volume_down', 'play_radio', 'cooking_query', 'datetime_convert', 'qa_maths', 'iot_hue_lightoff', 'iot_hue_lighton', 'transport_query', 'music_likeness', 'email_query', 'play_music', 'audio_volume_mute', 'social_post', 'alarm_set', 'qa_factoid', 'calendar_set', 'play_game', 'alarm_remove', 'lists_remove', 'transpor

## Dataloaders [Optional]

- Instead of going through individual samples in the dataset, we would like to 
batches of samples to train our model. 
- Dataloader
    1. Creates an "iterator" over the dataset, which returns a batch of samples every turn
    2. Handles shuffling
    3. Collates samples into batches, by padding the input sequences to the maximum length in the batch

In [8]:
# from torch.utils.data import DataLoader

# train_dataloader = DataLoader(
#     tokenized_datasets["train"], shuffle=True, batch_size=8, collate_fn=data_collator
# )
# eval_dataloader = DataLoader(
#     tokenized_datasets["validation"], batch_size=8, collate_fn=data_collator
# )

In [9]:
# Check the shape of the tensors
# for batch in tqdm(train_dataloader):
#     break
# {k: v.shape for k, v in batch.items()}

```python
{'labels': torch.Size([8]),
 'input_ids': torch.Size([8, 25]),
 'attention_mask': torch.Size([8, 25])}
```

## Load model using an `AutoModel`

In [10]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=tokenized_datasets['train'].features['labels'].num_classes)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.bias', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'classifier.weight', 'classifier.bias', 'pre_classifier.

In [11]:
model

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

In [12]:
from peft import get_peft_config, get_peft_model, LoraConfig, TaskType

peft_config = LoraConfig(
    task_type=TaskType.SEQ_CLS, 
    inference_mode=False, 
    r=16, 
    lora_alpha=16, 
    lora_dropout=0.1, 
    bias="none", 
    target_modules=["q_lin", "k_lin", "v_lin", "out_lin", "pre_classifier", "lin1", "lin2", "classifier"]
)

# model = AutoModelForSeq2SeqLM.from_pretrained(model_name_or_path)
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

trainable params: 2,676,216 || all params: 69,039,096 || trainable%: 3.876377523830845


In [13]:
model.modules_to_save

{'classifier', 'score'}

In [14]:
# # Let's check the output of a single forward pass
# outputs = model(**batch)
# print(outputs.loss, outputs.logits.shape)

## Setup Evaluation Metric

In [ ]:
import numpy as np
import evaluate

metric_acc = evaluate.load("accuracy")
metric_f1 = evaluate.load("f1")

# We need to define a compute_metric function that is supported by the Trainer output
# It basically converts the logits to predictions and then calls the metric
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    print("Predictions: ", predictions)
    print("Labels: ", labels)
    return {
        "accuracy": metric_acc.compute(predictions=predictions, references=labels), 
        "micro_f1": metric_f1.compute(predictions=predictions, references=labels, average="micro"),
        "macro_f1": metric_f1.compute(predictions=predictions, references=labels, average="macro")
    }

In [47]:
compute_metrics((
    [
        [0.1, 0.5, 0.4], 
        [0.3, 0.4, 0.3], 
        [0.2, 0.6, 0.2], 
        [0.1, 0.2, 0.7], 
        [0.2, 0.5, 0.3]
    ],
    [0, 1, 0, 2, 1] 
))

Predictions:  [1 1 1 2 1]
Labels:  [0, 1, 0, 2, 1]


{'accuracy': {'accuracy': 0.6},
 'micro_f1': {'f1': 0.6},
 'macro_f1': {'f1': 0.5555555555555555}}

In [17]:
train_data_subset = tokenized_datasets["train"] # .select(range(40960))
eval_data_subset = tokenized_datasets["validation"] #.select(range(2096))

## Setup Trainer and Finetuning

1. When testing your code for the first time, it is a better idea to use a smaller dataset,
so that you can quickly iterate over your code.
2. We have done the same here by using 128 + 32 samples from the dataset, for training and validation respectively.

In [18]:
from transformers import TrainingArguments
eval_steps = 5000
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=10,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    # warmup_steps=500,
    # weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    evaluation_strategy="steps",
    save_strategy="steps",
    save_steps=eval_steps,
    eval_steps=eval_steps,
    load_best_model_at_end=True,
    metric_for_best_model="micro_f1",
    greater_is_better=True,
    report_to="none" # set "wandb" here to log to wandb
    # report_to="wandb"
)

**Wandb Logging**

When training ML models, keeping track of various hyperparameters, the training and validation losses, and evaluation metrics is very important.
`wandb` can help you here.

To install wandb, run following commands in your terminal:

```bash
pip install wandb
# Then login to your wandb account
wandb login
```

Once, that is done, in TrainingArguments, set the `report_to` argument to `wandb`.

In [19]:
from transformers import Trainer

trainer = Trainer(
    model,
    training_args,
    # train_dataset=tokenized_datasets["train"],
    # eval_dataset=tokenized_datasets["validation"],
    train_dataset=train_data_subset, # Using the smaller subsets
    eval_dataset=eval_data_subset,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [20]:
stats = trainer.train()

/home/bsantra/miniconda3/envs/py38/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 587214
  Num Epochs = 10
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 183510
  Number of trainable parameters = 2676216
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: kgp-nlp (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.15.8 is available!  To

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss,Accuracy
5000,2.872100,2.825798,0.269437
10000,2.647600,2.509979,0.341329
15000,2.361500,2.318371,0.389910
20000,2.338300,2.178634,0.421332
25000,2.034900,2.095860,0.445116
30000,2.066500,2.022863,0.461956
35000,2.242900,1.945200,0.485210
40000,1.819900,1.895621,0.497488
45000,1.976000,1.862018,0.508097
50000,1.998600,1.812141,0.521812


***** Running Evaluation *****
  Num examples = 103683
  Batch size = 32
Saving model checkpoint to ./results/checkpoint-5000
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
tokenizer config file saved in ./results/checkpoint-5000/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-5000/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 103683
  Batch size = 32
Saving model checkpoint to ./results/checkpoint-10000
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
tokenizer config file saved in ./results/checkpoint-10000/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-10000/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 103683
  Batch size = 32
Saving model checkpoint to ./results/checkpoint-15000
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
tokenizer config file saved in ./results/checkpoint-15000/tokenizer_config.json
Special toke

In [49]:
trainer.compute_metrics = compute_metrics

In [50]:
# Print Stats
from pprint import pprint
pprint(trainer.evaluate())

***** Running Evaluation *****
  Num examples = 103683
  Batch size = 32


Predictions:  [40 31  1 ... 50 13 13]
Labels:  [40 31 31 ... 33 44 17]
{'epoch': 10.0,
 'eval_accuracy': {'accuracy': 0.610726927268694},
 'eval_loss': 1.5062118768692017,
 'eval_macro_f1': {'f1': 0.5647782620578362},
 'eval_micro_f1': {'f1': 0.610726927268694},
 'eval_runtime': 45.0682,
 'eval_samples_per_second': 2300.58,
 'eval_steps_per_second': 71.913}
